In [ ]:
from pydantic import create_model
from langchain_groq import ChatGroq
import inspect, json, os, ast
from IPython.display import display, Markdown
from inspect import Parameter
from crewai import Agent, Task, Crew
from langchain.agents.tools import tool # same thing as from crewai_tools import tool
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model="llama3-70b-8192",
    temperature=1,
)

def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None
    
    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)


@tool
def python(code: str):
    """Returns the result of executing 'code' using Python."""
    try:
        result = run(code)
        return result
    except Exception as e:
        return str(e)

code_interpreter = Agent(
    role = "Code Interpreter",
    goal = "Solving problems by writing and executing Python code.",
    backstory = """You are a coding LLM that writes and executes Python code, then 
    returns the result of the last expression as if it were executed in a Jupyter 
    notebook environment. Do not use print statements, and do not print out the 
    evaluation in any form. Simply return the result of the final expression 
    evaluated. Ensure that the output is only the result of the code execution.""",
    verbose=True,
    allow_delegation=False,
    tools = [python],
    llm = llm,
)

problem = Task(
    description = "{prompt}",
    expected_output = """The result after executing the Python code.""",
    agent = code_interpreter,
)

crew = Crew(
    agents = [code_interpreter],
    tasks = [problem],
)

result = crew.kickoff(inputs={"prompt":input()})